In [1]:
import pandas as pd
import os
import numpy as np
import time

In [2]:
############################################################
#seleziona anno
anno = 2020
############################################################

In [3]:
fatture = pd.read_csv(os.getcwd()+'\\FATTURE CLIENTI ALLPLEX 2018-2019 PER SAECON.csv', 
                       sep = ';', names=['CODART', 'CODCLIFOR', 'RAGSOC', 'DITTA', 'SEZ', 
                                         'NUMDOC', 'DATA', 'MESE', 'PROGRIGA', 
                                         'QTA', 'IMPORTO', 'CODCOMMESSA'], decimal='.')
fatture['DATAD'] = pd.to_datetime(fatture['DATA'], format='%d/%m/%Y')
fatture['ANNO'] = fatture['DATAD'].dt.year
fatture = fatture[fatture['ANNO'] == anno]

In [4]:
# prima pivot fatture
fatture_p = fatture.groupby(['CODART', 'ANNO', 'MESE'])['QTA', 'IMPORTO'].sum().reset_index()
fatture_p['FLAG'] = 0

In [5]:
ore = pd.read_csv(os.getcwd()+'\ORE TOTALI mazak ALLPLEX 2018-2019.csv', 
                       sep = ';', decimal='.')
ore = ore[ore['ANNO'] == anno]

In [6]:
#prima pivot ore
ore_p = ore.groupby(['ANNO', 'MESE', 'CODICEARTICOLO'])['DATA'].count().reset_index()

In [7]:
#####    trasforma gli SL in PF    ####
# ore_p['CODICEARTICOLO'] = ore_p['CODICEARTICOLO'].apply(lambda x: "PF"+x[2:] if x.startswith('SL') else x)

#####    togli gli SL e PF    ####
# ore_p['CODICEARTICOLO'] = ore_p['CODICEARTICOLO'].apply(lambda x: x[2:] if x.startswith('SL') or x.startswith('PF') else x)
# fatture_p['CODART'] = fatture_p['CODART'].apply(lambda x: x[2:] if x.startswith('SL') or x.startswith('PF') else x)

#prima pivot ore
ore_to_exp = ore.groupby(['ANNO', 'MESE', 'CODICEARTICOLO'])['ORE', 'MINUTI', 'SECONDI'].sum().reset_index()

path = os.path.join("MAZAK\\",str(anno))

ore_to_exp.to_csv(path+"\\ore_mazak_2019.csv", decimal = ',', index=False)

In [8]:
def calcola(db, fatt_p):
    finale = pd.DataFrame(columns=['CODICEARTICOLO', 'ANNO_NICIM', 'MESE_NICIM', 'MESE_FATTURA', 'QTA', 'FATTURA'])
    
    for index, row in db.iterrows():

        articolo_nicim = row['CODICEARTICOLO'].strip()
        anno_nicim = row['ANNO']
        mese_nicim = row['MESE']


        foglio_temp = db[db['CODICEARTICOLO'].str.strip() == articolo_nicim]
        mese_limite = 0

        #cicla la lista degli articoli in nicim
        for index1, row1 in foglio_temp.iterrows():

            # se articolo è stato fatto in un altro mese rispetto a quello dell analisi
            # allora associa alla variabile l articolo in questione
            if row1['MESE'] > mese_nicim:
                mese_limite = row1['MESE']
                break    

            if mese_limite == 0:
                # senno la variabile mese diventa quella dell unico articolo in analisi
                mese_limite = 12



        mese_fatt_1 = 0
        anno_fatt = 0
        qta_fatt = 0
        fatt_fatt = 0


        fatture_filtr = fatt_p[fatt_p['CODART'].str.strip() == articolo_nicim]


        for index2, row2 in fatture_filtr.iterrows():

            anno_fatt = row2['ANNO']
            mese_fatt = row2['MESE']
            flag = row2['FLAG']

            if mese_fatt >= mese_nicim and mese_fatt < mese_limite and flag == 0:

                #mese da fattura
                mese_fatt_1 = row2['MESE']

                #qta da fattura
                qta_fatt = row2['QTA'] + qta_fatt

                #importo da fattura
                fatt_fatt = row2['IMPORTO'] + fatt_fatt

                fatt_p.iloc[index2, fatt_p.columns.get_loc('FLAG')] = 1
                fatture_filtr.at[index2, 'FLAG'] = 1


            if mese_fatt >= mese_nicim and mese_fatt == 12 and mese_limite == 12 and flag == 0:

                #mese da fattura
                mese_fatt_1 = row2['MESE']

                #qta da fattura
                qta_fatt = row2['QTA'] + qta_fatt

                #importo da fattura
                fatt_fatt = row2['IMPORTO'] + fatt_fatt

                fatt_p.iloc[index2, fatt_p.columns.get_loc('FLAG')] = 1  
                fatture_filtr.at[index2, 'FLAG'] = 1



        #incrocio articoli non trovati sostituendo SL con PF

        if articolo_nicim.startswith('SL') and fatt_fatt == 0:

            articolo_nicim = 'PF'+articolo_nicim[2:]

            fatture_filtr = fatt_p[fatt_p['CODART'].str.strip() == articolo_nicim]

            for index2, row2 in fatture_filtr.iterrows():

                anno_fatt = row2['ANNO']
                mese_fatt = row2['MESE']
                flag = row2['FLAG']

                if mese_fatt >= mese_nicim and mese_fatt < mese_limite and flag == 0:

                    #mese da fattura
                    mese_fatt_1 = row2['MESE']

                    #qta da fattura
                    qta_fatt = row2['QTA'] + qta_fatt

                    #importo da fattura
                    fatt_fatt = row2['IMPORTO'] + fatt_fatt

                    fatt_p.iloc[index2, fatt_p.columns.get_loc('FLAG')] = 1
                    fatture_filtr.at[index2, 'FLAG'] = 1


                if mese_fatt >= mese_nicim and mese_fatt == 12 and mese_limite == 12 and flag == 0:

                    #mese da fattura
                    mese_fatt_1 = row2['MESE']

                    #qta da fattura
                    qta_fatt = row2['QTA'] + qta_fatt

                    #importo da fattura
                    fatt_fatt = row2['IMPORTO'] + fatt_fatt

                    fatt_p.iloc[index2, fatt_p.columns.get_loc('FLAG')] = 1  
                    fatture_filtr.at[index2, 'FLAG'] = 1



        finale = finale.append(pd.Series([articolo_nicim, anno_nicim, mese_nicim, 
                                         mese_fatt_1, qta_fatt, fatt_fatt], index=finale.columns ), ignore_index=True) 
        

    return finale


In [9]:
def calcola_macch(db, fatt_p):
    
    finale_2 = pd.DataFrame(columns=['CODICEARTICOLO', 'ANNO_NICIM', 'MESE_NICIM', 'MESE_FATTURA', 'QTA', 'FATTURA', 'MACCHINA',
                                  'ORE', 'MIN', 'SEC'])
    
    for index, row in db.iterrows():

        articolo_nicim = row['CODICEARTICOLO'].strip()
        anno_nicim = row['ANNO']
        mese_nicim = row['MESE']
        macchina_nicim = row['MACCHINARIO']
        ore_nicim = row['ORE']
        min_nicim = row['MINUTI']
        sec_nicim = row['SECONDI']


        foglio_temp = db[db['CODICEARTICOLO'].str.strip() == articolo_nicim]
        mese_limite = 0

        #cicla la lista degli articoli in nicim
        for index1, row1 in foglio_temp.iterrows():

            # se articolo è stato fatto in un altro mese rispetto a quello dell analisi
            # allora associa alla variabile l articolo in questione
            if row1['MESE'] > mese_nicim:
                mese_limite = row1['MESE']
                break    

            if mese_limite == 0:
                # senno la variabile mese diventa quella dell unico articolo in analisi
                mese_limite = 12



        mese_fatt_1 = 0
        anno_fatt = 0
        qta_fatt = 0
        fatt_fatt = 0


        fatture_filtr = fatt_p[fatt_p['CODART'].str.strip() == articolo_nicim]


        for index2, row2 in fatture_filtr.iterrows():

            anno_fatt = row2['ANNO']
            mese_fatt = row2['MESE']
            flag = row2['FLAG']

            if mese_fatt >= mese_nicim and mese_fatt < mese_limite and flag == 0:

                #mese da fattura
                mese_fatt_1 = row2['MESE']

                #qta da fattura
                qta_fatt = row2['QTA'] + qta_fatt

                #importo da fattura
                fatt_fatt = row2['IMPORTO'] + fatt_fatt

                fatt_p.iloc[index2, fatt_p.columns.get_loc('FLAG')] = 1
                fatture_filtr.at[index2, 'FLAG'] = 1


            if mese_fatt >= mese_nicim and mese_fatt == 12 and mese_limite == 12 and flag == 0:

                #mese da fattura
                mese_fatt_1 = row2['MESE']

                #qta da fattura
                qta_fatt = row2['QTA'] + qta_fatt

                #importo da fattura
                fatt_fatt = row2['IMPORTO'] + fatt_fatt

                fatt_p.iloc[index2, fatt_p.columns.get_loc('FLAG')] = 1  
                fatture_filtr.at[index2, 'FLAG'] = 1



        #incrocio articoli non trovati sostituendo SL con PF

        if articolo_nicim.startswith('SL') and fatt_fatt == 0:

            articolo_nicim = 'PF'+articolo_nicim[2:]

            fatture_filtr = fatt_p[fatt_p['CODART'].str.strip() == articolo_nicim]

            for index2, row2 in fatture_filtr.iterrows():

                anno_fatt = row2['ANNO']
                mese_fatt = row2['MESE']
                flag = row2['FLAG']

                if mese_fatt >= mese_nicim and mese_fatt < mese_limite and flag == 0:

                    #mese da fattura
                    mese_fatt_1 = row2['MESE']

                    #qta da fattura
                    qta_fatt = row2['QTA'] + qta_fatt

                    #importo da fattura
                    fatt_fatt = row2['IMPORTO'] + fatt_fatt

                    fatt_p.iloc[index2, fatt_p.columns.get_loc('FLAG')] = 1
                    fatture_filtr.at[index2, 'FLAG'] = 1


                if mese_fatt >= mese_nicim and mese_fatt == 12 and mese_limite == 12 and flag == 0:

                    #mese da fattura
                    mese_fatt_1 = row2['MESE']

                    #qta da fattura
                    qta_fatt = row2['QTA'] + qta_fatt

                    #importo da fattura
                    fatt_fatt = row2['IMPORTO'] + fatt_fatt

                    fatt_p.iloc[index2, fatt_p.columns.get_loc('FLAG')] = 1  
                    fatture_filtr.at[index2, 'FLAG'] = 1



        finale_2 = finale_2.append(pd.Series([articolo_nicim, anno_nicim, mese_nicim, 
                                     mese_fatt_1, qta_fatt, fatt_fatt, macchina_nicim,
                                     ore_nicim, min_nicim, sec_nicim], index=finale_2.columns), ignore_index=True)    
        

    return finale_2


In [10]:
######## algoritmo ##########
start = time.time()

finale = calcola(ore_p, fatture_p)
    
end = time.time()
print(end - start)

154.27108764648438


In [11]:
path = os.path.join("MAZAK\\",str(anno))

finale.to_csv(path+"\\mazak_no_macch.csv", decimal = ',', index=False)

In [12]:
#### lista pantografi
ore_pantografi = pd.read_csv(os.getcwd()+'\ORE TOTALI pantografi ALLPLEX 2018-2019.csv', 
                       sep = ';', decimal='.')
ore_pantografi = ore_pantografi[ore_pantografi['ANNO'] == anno]

#prima pivot ore
ore_m_p = ore_pantografi.groupby(['ANNO', 'MESE', 'CODICEARTICOLO'])['DATA'].count().reset_index()

#elimina spazi codice articolo
ore_m_p['CODICEARTICOLO'] = ore_m_p['CODICEARTICOLO'].str.strip()

In [13]:
#####    trasforma gli SL in PF    ####
# ore_p['CODICEARTICOLO'] = ore_p['CODICEARTICOLO'].apply(lambda x: "PF"+x[2:] if x.startswith('SL') else x)

#####    togli gli SL e PF    ####
#ore_m_p['CODICEARTICOLO'] = ore_m_p['CODICEARTICOLO'].apply(lambda x: x[2:] if x.startswith('SL') or x.startswith('PF') else x)

In [14]:
#esporta tabella
ore_m_p.to_csv(path+"\\pantografi.csv", decimal = ',', index=False)

In [15]:
###########################DIVISIONE PER MACCHINA#############################
fatture = pd.read_csv(os.getcwd()+'\FATTURE CLIENTI ALLPLEX 2018-2019 PER SAECON.csv', 
                       sep = ';', names=['CODART', 'CODCLIFOR', 'RAGSOC', 'DITTA', 'SEZ', 
                                         'NUMDOC', 'DATA', 'MESE', 'PROGRIGA', 
                                         'QTA', 'IMPORTO', 'CODCOMMESSA'], decimal='.')
fatture['DATAD'] = pd.to_datetime(fatture['DATA'], format='%d/%m/%Y')
fatture['ANNO'] = fatture['DATAD'].dt.year
fatture = fatture[fatture['ANNO'] == anno]

In [16]:
# prima pivot fatture
fatture_p = fatture.groupby(['CODART', 'ANNO', 'MESE'])['QTA', 'IMPORTO'].sum().reset_index()
fatture_p['FLAG'] = 0

In [17]:
ore = pd.read_csv(os.getcwd()+'\ORE TOTALI mazak ALLPLEX 2018-2019.csv', 
                       sep = ';', decimal='.')
ore = ore[ore['ANNO'] == anno]

In [18]:
ore.head()

,DITTA,DATA,ANNO,MESE,ORE,MINUTI,SECONDI,MACCHINARIO,CODICEARTICOLO
0,2,28/01/2020,2020,1,0,14,15,MAZAK2015,PF81K60104
1,2,28/01/2020,2020,1,0,18,0,MAZAK2015,PF81K60104
3,2,28/01/2020,2020,1,0,6,5,MAZAK2015,PF81K60104
1622,2,13/06/2020,2020,6,0,21,40,MAZAKJ600,PFALLPLEXPMMACC
1624,2,01/02/2020,2020,2,0,40,46,MAZAKVTC800,PFALLPLEXPMMACC


In [19]:
#prima pivot ore
ore_p = ore.groupby(['ANNO', 'MESE', 'CODICEARTICOLO', 'MACCHINARIO'])['ORE', 'MINUTI', 'SECONDI'].sum().reset_index()

In [20]:
#####    trasforma gli SL in PF    ####
# ore_p['CODICEARTICOLO'] = ore_p['CODICEARTICOLO'].apply(lambda x: "PF"+x[2:] if x.startswith('SL') else x)

#####    togli gli SL e PF    ####
#ore_p['CODICEARTICOLO'] = ore_p['CODICEARTICOLO'].apply(lambda x: x[2:] if x.startswith('SL') or x.startswith('PF') else x)
#fatture_p['CODART'] = fatture_p['CODART'].apply(lambda x: x[2:] if x.startswith('SL') or x.startswith('PF') else x)

In [21]:
######## algoritmo ##########
start = time.time()

finale_2=calcola_macch(ore_p, fatture_p)  

end = time.time()
print(end - start)

117.9780650138855


In [22]:
finale_2.to_csv(path+"\\Mazak_macch.csv", decimal = ',', index=False)

In [23]:
#fatture anno dopo

anno = anno + 1

fatture = pd.read_csv(os.getcwd()+'\FATTURE CLIENTI ALLPLEX 2018-2019 PER SAECON.csv', 
                       sep = ';', names=['CODART', 'CODCLIFOR', 'RAGSOC', 'DITTA', 'SEZ', 
                                         'NUMDOC', 'DATA', 'MESE', 'PROGRIGA', 
                                         'QTA', 'IMPORTO', 'CODCOMMESSA'], decimal='.')
fatture['DATAD'] = pd.to_datetime(fatture['DATA'], format='%d/%m/%Y')
fatture['ANNO'] = fatture['DATAD'].dt.year
fatture = fatture[fatture['ANNO'] == anno]

fatture_p = fatture.groupby(['CODART', 'ANNO', 'MESE'])['QTA', 'IMPORTO'].sum().reset_index()
fatture_p['CODART'] = fatture_p['CODART'].astype(str).str.strip()

In [24]:
#####    trasforma gli SL in PF    ####
# ore_p['CODICEARTICOLO'] = ore_p['CODICEARTICOLO'].apply(lambda x: "PF"+x[2:] if x.startswith('SL') else x)

#####    togli gli SL e PF    ####
#fatture_p['CODART'] = fatture_p['CODART'].apply(lambda x: x[2:] if x.startswith('SL') or x.startswith('PF') else x)

In [25]:
path = os.path.join("MAZAK\\",str(anno-1))

fatture_p.to_csv(path+"\\fatture anno dopo.csv", decimal = ',', index=False)

In [26]:
#ore anno dopo

ore = pd.read_csv(os.getcwd()+'\ORE TOTALI mazak ALLPLEX 2018-2019.csv', 
                       sep = ';', decimal='.')
ore = ore[ore['ANNO'] == anno]

ore_p = ore.groupby(['ANNO', 'MESE', 'CODICEARTICOLO'])['DATA'].count().reset_index()

ore_p['CODICEARTICOLO'] = ore_p['CODICEARTICOLO'].astype(str).str.strip()

In [27]:
#####    trasforma gli SL in PF    ####
# ore_p['CODICEARTICOLO'] = ore_p['CODICEARTICOLO'].apply(lambda x: "PF"+x[2:] if x.startswith('SL') else x)

#####    togli gli SL e PF    ####
#ore_p['CODICEARTICOLO'] = ore_p['CODICEARTICOLO'].apply(lambda x: x[2:] if x.startswith('SL') or x.startswith('PF') else x)

In [28]:
path = os.path.join("MAZAK\\",str(anno-1))

ore_p.to_csv(path+"\\ore anno dopo.csv", decimal = ',', index=False)